In [6]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 이미지 로드 (또는 비디오/웹캠 프레임 사용)
image = cv2.imread('D:\\human_fall\\train_origin\\Y\\00004_H_A_SY_C1_I001.jpg')

# YOLO로 사람 감지
results = yolo_model(image)

for r in results:
    boxes = r.boxes
    for box in boxes:
        # 사람 클래스인 경우에만 처리 (YOLO 클래스 인덱스에 따라 조정 필요)
        if int(box.cls) == 0:  # 0은 보통 'person' 클래스
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            
            # 바운딩 박스 추출
            person_image = image[y1:y2, x1:x2]
            
            # MediaPipe로 포즈 추정
            results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
            
            if results_pose.pose_landmarks:
                # 원본 이미지에 포즈 그리기
                mp_drawing.draw_landmarks(
                    image[y1:y2, x1:x2],
                    results_pose.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS
                )
            
            # 바운딩 박스 그리기
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# 결과 표시
cv2.namedWindow("test", cv2.WINDOW_NORMAL)
cv2.resizeWindow("test", 1920, 1080)
cv2.imshow("test", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 1 Non_Fall, 3.5ms
Speed: 2.0ms preprocess, 3.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
